In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
sns.set()

%matplotlib inline
sns.set()


plt.tight_layout()  # to fit the text overlap in the graph --

<Figure size 640x480 with 0 Axes>

In [2]:
pd.set_option('display.max_columns', 1000) # to see all columns data
pd.set_option('display.max_rows', 50) # to see all columns data
#pd.set_option('display.float_format', '{:.2%}'.format)
#pd.options.display.float_format = None
pd.set_option("display.max_colwidth", -1) # display the full text

/var/folders/q0/xtygmwwj1xd7jjkr58wmvp_80000gn/T/ipykernel_3900/3729420087.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1) # display the full text


## Exploring the Date (EDA) ------

In [ ]:
df= pd.read_excel("Salaries.xlsx")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe().round(2)

In [ ]:
df.nunique()

In [ ]:
# Check the nall values --------

df.isnull().sum()

In [ ]:
df.set_index('Id')

In [ ]:
# Check the status column -------

df['Status'].isnull().sum()

In [ ]:
df['Status'].nunique()


In [ ]:
df['Status'].value_counts()

### I will drop these columns: 

Notes column [is empty ]

Agency [is all values San Francisco] 

and Status [there are 110535 null values and it doesn't help ]



In [ ]:
df.drop(["Notes", "Agency","Status"],axis=1, inplace=True)

In [ ]:
df.set_index('Id')

In [ ]:
df.nunique()

In [ ]:
# Check id duplicate for confirm not duplicate names, and don't same values ----

duplicate_id= df[df.duplicated('Id')]
duplicate_id

I notes there are some rows contains [Not provided] values let to check

In [ ]:
df[df.EmployeeName == 'Not provided']

In [ ]:
# I will drop these 4 rows by id from the data ---

df.drop([148646,148650,148651,148652],inplace=True)

after check the [0] values in ['BasePay'] column let keep it with [0] values

In [ ]:
df[df['BasePay']==0]

#### Check the the TotalPayBenefits == 0 these vlaues don't help in our data so I will remove them

In [ ]:
df[df['TotalPayBenefits']==0]

In [ ]:
# Drop all values in TotalPayBenefits column == 0 ---- 

df.drop(df[df.TotalPayBenefits == 0].index, inplace=True)

#### Convert data types date / BasePay/OvertimePay/OtherPay/Benefits

In [ ]:
# Convert object to float ---- 

df['BasePay'] = df['BasePay'].astype(float)
df['OvertimePay'] = df['OvertimePay'].astype(float)
df['OtherPay'] = df['OtherPay'].astype(float)
df['Benefits'] = df['Benefits'].astype(float)

In [ ]:
df.info()

In [ ]:
# The highest 50 Most Common Jobs ----

df['JobTitle'].value_counts().head(50)

In [ ]:
df['JobTitle'].value_counts().head(50).plot(kind='bar',figsize=(15, 6), title='# # The highest 50 Most Common Jobs',
                                            ylabel='numbers of jobs', xlabel='Jobs')

In [ ]:
# The lowest 50 Most Common Jobs  ----

df['JobTitle'].value_counts().tail(50)

In [ ]:
df['JobTitle'].value_counts().tail(50).plot(kind='bar',figsize=(15, 6), title='The lowest 50 Most Common Jobs',
                                            ylabel='numbers of jobs', xlabel='Jobs')

In [ ]:
# Sort highest benefits pay by year -----

highest_benefits_yeraly=df.groupby("Year")["TotalPayBenefits"].sum().sort_values(ascending=False)

In [ ]:
highest_benefits_yeraly.plot()

In [ ]:
highest_benefits_yeraly=df.groupby("Year")["TotalPayBenefits"].sum().plot(kind='bar',figsize=(10, 6), 
                                                                title='Sort highest benefits pay by year',
                                                                 ylabel='Highest benefits', xlabel='Year',color="green")

In [ ]:
df.set_index('Id')

In [ ]:
df.head(10)

In [ ]:
# Top 20 Employee with their jobs titles gain a TotalPlay Benefits ------

df.nlargest(20,'TotalPayBenefits')

In [ ]:
# Select highest employees have Total Pay Benefits----

df.nlargest(10,'TotalPayBenefits').plot(kind='bar',figsize=(10, 6), title='Top 10 Employees with their jobs titles gain a TotalPlay Benefit',
                                            ylabel='Total Pay Benefits', xlabel='Id number')

In [ ]:
# # The Maximum Basic salary --------

df.nlargest(20,'BasePay')

I notice that it doesn't necessarily mean the highest basic salary takes the highest Total Pay Benefits

In [ ]:
# Numbers of jobs ----- 

df.groupby("JobTitle").agg("count").sort_values(by= "BasePay", ascending=False).head(20)

In [ ]:
# numbers of employees per year-----

df.groupby("Year").agg({'Id': "count"}).sort_values(by= "Year", ascending=False).head(20)

In [ ]:
df.groupby("Year").agg({'Id': "count"}).sort_values(by= "Year", ascending=False).plot(kind='bar',figsize=(10, 6), title='Numbers of jobs (Employees) per year',
                                            ylabel='Numbers of Employees', xlabel='Years')

In [ ]:
# Numbers of  the jobs per year ------

df.groupby(["Year",'JobTitle']).agg({'JobTitle': "count"}).sort_values(by= "Year", ascending=False)

In [ ]:
# The 30 highest Bisc Pay and TotalPayBenefits jobs -----

df.groupby("JobTitle")['Year',"BasePay","TotalPayBenefits"].max().sort_values(by='BasePay', ascending=False).head(30)

In [ ]:
# the mean of BasePay and TotalPayBenefits per year ----

Bsic_TotalPayBenefits_mean=df.pivot_table(index=['Year'], values=['TotalPayBenefits','BasePay']).round(2)

Bsic_TotalPayBenefits_mean

In [ ]:
Bsic_TotalPayBenefits_mean.plot(kind='bar',figsize=(10, 6), title='Comper between the mean of Basic and total benefits pay per year',
                                            ylabel='Basic and Total Pay Benefits', xlabel='Years')

In [ ]:
# The highest year spending overtime --------

ss=df.groupby("Year")["OvertimePay"].sum().sort_values(ascending=False)
ss

In [ ]:
 df.groupby("Year")["OvertimePay"].sum().sort_values(ascending=False).plot(kind='bar',figsize=(10, 6), 
                                                                    title='The highest year spending the overtime',
                                                                    ylabel='Overtime payment', xlabel='Years',
                                                                    color=['cyan', 'red', 'green', 'blue', 'cyan'])

## Thank you 